In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

%reload_ext tensorboard

time: 299 ms (started: 2021-09-03 14:21:56 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #4 Sept 03, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main e1f5a28] 更新 #4 Sept 03, 2021
 5 files changed, 1679 insertions(+), 906 deletions(-)
 create mode 100644 "Lightning-Flash/.ipynb_checkpoints/\344\273\216FLASH\345\210\260LIGHTNING-checkpoint.ipynb"
 create mode 100644 "Lightning-Flash/\344\273\216FLASH\345\210\260LIGHTNING.ipynb"
 rewrite "Lightning-Flash/\345\277\253\351\200\237\345\274\200\345\247\213.ipynb" (75%)


To github.com:ustchope/pytorch_lightning_study.git
   2159ffa..e1f5a28  main -> main


time: 3.99 s (started: 2021-09-03 14:20:04 +08:00)


In [2]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from collections import OrderedDict

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer, LightningDataModule
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
import torchmetrics
import torchvision.transforms as transforms

seed_everything(7)

PATH_DATASETS = os.environ.get('PATH_DATASETS', '.')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

Global seed set to 7


time: 1.66 s (started: 2021-09-03 14:21:59 +08:00)


Flash 建立在 PyTorch Lightning 之上，以抽象出不必要的样板：
* 数据科学
* Kaggle
* 业务用例
* 应用研究

Flash 是一个高级库，Lightning 是一个低级库。
* Flash（高级）
* Lightning（中级）
* PyTorch（低级）

随着复杂性的增加或减少，用户可以在 Flash 和 Lightning 之间无缝切换，以找到适合他们的抽象级别。

|方法| 灵活性 | 最低深度学习专业水平 | PyTorch 知识|  用例|
|----|----|----|----|----|
|使用开箱即用的任务|低|新手+|低+|快速基线、数据科学、分析、应用研究|
|使用通用任务|中等|中级+|中级+|快速基线，数据科学|
|构建自定义任务|高|中级+|中级+|快速基线、自定义业务背景、应用研究|
|构建LightningModule|Ultimate（有组织的 PyTorch）|专家+|专家+|对于你可以用 PyTorch 做的任何事情，人工智能研究（学术和企业）|

# 使用开箱即用的任务

任务可以来自多个地方：
* Flash
* 其他基于 Lightning 的库
* 你自己的库

使用任务几乎需要对深度学习和 PyTorch 的零知识。 重点是尽快解决问题。 这非常适合：
* 数据科学
* 分析
* 应用研究

# 使用通用任务

如果遇到没有匹配任务的问题，可以使用通用任务。 然而，这确实需要一些 PyTorch 知识，但不需要很多关于深度学习所有细节的知识。

这非常适合：
* 数据科学
* Kaggle 基线
* 快速基线
* 应用研究
* 学习深度学习

> 如果你使用过像 Keras 这样的东西，这是最相似的抽象级别。

# 构建自定义任务

如果您喜欢冒险并且没有针对特定应用问题的现成任务，请考虑构建您自己的任务。 这需要相当数量的 PyTorch 知识，但不会太多，因为任务是 LightningModules，已经为你抽象了很多细节。

这非常适合：
* 数据科学
* 研究人员为企业数据科学团队建设
* 应用研究
* 自定义业务上下文

> 在公司环境中，这里的一个好的设置是拥有自己的类似 Flash 的库，其中包含与您的业务问题相关的任务。

# 构建闪电模块

一旦您达到 Flash 提供的灵活性阈值，就可以直接转向 LightningModule。 LightningModule 由 PyTorch 组织，但为您提供相同的灵活性。 但是，您必须已经相当了解 PyTorch，并且至少熟悉基本的深度学习概念。

这非常适合：
* 专家
* 学术人工智能研究
* 企业人工智能研究
* 先进的应用研究
* 发表论文